<a href="https://colab.research.google.com/github/Domdec32/IMLO/blob/main/IMLO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import torch
import torch.optim as optim
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
import torch.nn.functional as F
from torch.utils.data import ConcatDataset
from torch.utils.data import TensorDataset

In [14]:
ImageWidth = 200
ImageHeight = 200
batch_size = 64
LearningRate = 0.001
NumberofEpochs = 30

In [15]:
transform = transforms.Compose([
    transforms.Resize((ImageHeight, ImageWidth)),
    transforms.ToTensor()
])

augment_transform = transforms.Compose([
    transforms.Resize((ImageHeight, ImageWidth)),
    transforms.RandomRotation(degrees=20),
    transforms.RandomHorizontalFlip(),
    transforms.RandomResizedCrop((ImageHeight, ImageWidth), scale=(0.8, 1.0)),
    transforms.ToTensor()
])

In [16]:

training_data = datasets.Flowers102(
    root="data",
    split="train",
    download=True,
    transform=transform,
)

augmented_training_data1 = datasets.Flowers102(
    root="data",
    split="train",
    download=True,
    transform=augment_transform,
)

augmented_training_data2 = datasets.Flowers102(
    root="data",
    split="train",
    download=True,
    transform=augment_transform,
)

test_data = datasets.Flowers102(
    root="data",
    split="test",
    download=True,
    transform=transform,
)

evaluation_data = datasets.Flowers102(
    root="data",
    split="val",
    download=True,
    transform=transform,
)

In [17]:
combined_dataset1 = ConcatDataset([training_data, augmented_training_data1])
combined_dataset2 = ConcatDataset([combined_dataset1, augmented_training_data2])
train_dataloader = DataLoader(combined_dataset2, batch_size=batch_size, shuffle=True)

test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)
evaluation_dataloader = DataLoader(evaluation_data, batch_size=batch_size, shuffle=False)

In [18]:
for X, y in test_dataloader:
    print(f"Shape of X: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X: torch.Size([64, 3, 200, 200])
Shape of y: torch.Size([64]) torch.int64


In [19]:
class CNN(nn.Module):
    def __init__(self, ImageWidth, ImageHeight, num_classes=102):
        super(CNN, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=8, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc = nn.Linear(in_features = int(ImageWidth/4)*int(ImageHeight/4)*16, out_features=num_classes)

    def forward(self, x):
          x = self.pool(F.relu(self.conv1(x)))
          x = self.pool(F.relu(self.conv2(x)))
          x = self.fc(x.reshape(x.shape[0], -1))

          return x

model = CNN(ImageWidth, ImageHeight, num_classes=102)

In [20]:
x = torch.randn(32, 3, 200, 200)
y = model(x)
print(y.shape)

torch.Size([32, 102])


In [21]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

model = CNN(ImageWidth=ImageWidth, ImageHeight=ImageHeight, num_classes=102)
model = model.to(device)

criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr = LearningRate)

cuda


In [22]:
def check_accuracy(loader, model, device):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)

            scores = model(x)
            _, predictions = scores.max(1)

            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    model.train()
    accuracy = float(num_correct) / num_samples
    return accuracy

In [23]:
for epoch in range(NumberofEpochs):
  running_loss = 0
  for index, (x,y) in enumerate(train_dataloader):
    x = x.to(device)
    y = y.to(device)

    y_hat = model(x)
    loss = criterion(y_hat, y)
    running_loss += loss

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  print(f"Epoch {epoch}: loss: {running_loss}")
  eval_accuracy = check_accuracy(evaluation_dataloader, model, device)
  print(f'Evaluation Accuracy: {eval_accuracy * 100:.2f}%')

Epoch 0: loss: 208.7442626953125
Evaluation Accuracy: 9.51%
Epoch 1: loss: 150.8865509033203
Evaluation Accuracy: 18.82%
Epoch 2: loss: 96.31697082519531
Evaluation Accuracy: 25.49%
Epoch 3: loss: 64.93558502197266
Evaluation Accuracy: 26.18%
Epoch 4: loss: 48.76742172241211
Evaluation Accuracy: 28.92%
Epoch 5: loss: 38.26835250854492
Evaluation Accuracy: 28.53%
Epoch 6: loss: 35.39411926269531
Evaluation Accuracy: 29.02%
Epoch 7: loss: 28.15653419494629
Evaluation Accuracy: 32.65%
Epoch 8: loss: 24.235698699951172
Evaluation Accuracy: 31.27%
Epoch 9: loss: 22.753402709960938
Evaluation Accuracy: 32.06%
Epoch 10: loss: 20.664409637451172
Evaluation Accuracy: 31.37%
Epoch 11: loss: 18.753219604492188
Evaluation Accuracy: 32.75%
Epoch 12: loss: 17.291717529296875
Evaluation Accuracy: 31.86%
Epoch 13: loss: 17.636247634887695
Evaluation Accuracy: 30.69%
Epoch 14: loss: 14.211971282958984
Evaluation Accuracy: 31.86%
Epoch 15: loss: 12.939933776855469
Evaluation Accuracy: 33.04%
Epoch 16: l

In [24]:
#final test
test_accuracy = check_accuracy(test_dataloader, model, device)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

Test Accuracy: 30.17%
